In [2]:
!pip install dalle-pytorch

     |████████████████████████████████| 1.4 MB 15.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.7 MB 77.0 MB/s eta 0:00:01
     |████████████████████████████████| 804.1 MB 2.8 kB/s  eta 0:00:01     |███████████████▎                | 382.8 MB 48.6 MB/s eta 0:00:09     |██████████████████████████▉     | 673.8 MB 86.0 MB/s eta 0:00:02
     |████████████████████████████████| 45 kB 2.9 MB/s eta 0:00:01
     |████████████████████████████████| 64 kB 3.2 MB/s  eta 0:00:01
     |████████████████████████████████| 21.6 MB 52.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.9 MB 47.7 MB/s eta 0:00:01
     |████████████████████████████████| 806 kB 49.3 MB/s eta 0:00:01
     |████████████████████████████████| 274 kB 66.0 MB/s eta 0:00:01
     |████████████████████████████████| 111 kB 75.8 MB/s eta 0:00:01
     |████████████████████████████████| 10.6 MB 43.2 MB/s eta 0:00:01
     |████████████████████████████████| 4.2 MB 21.2 MB/s eta 0:00:01
     |█████████████████████████

In [7]:
!pip install wandb

     |████████████████████████████████| 1.8 MB 16.6 MB/s eta 0:00:01
     |████████████████████████████████| 131 kB 69.8 MB/s eta 0:00:01
  Using cached GitPython-3.1.17-py3-none-any.whl (166 kB)
     |████████████████████████████████| 97 kB 7.7 MB/s  eta 0:00:01
  Using cached gitdb-4.0.7-py3-none-any.whl (63 kB)
  Using cached smmap-4.0.0-py2.py3-none-any.whl (24 kB)
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21494 sha256=53b98e9a9397555f15dbc37a1f6d02441b12cd06be8c190f0c24eaf438b79139
  Stored in directory: /home/ec2-user/.cache/pip/wheels/59/9a/1d/3f1afbbb5122d0410547bf9eb50955f4a7a98e53a6d8b99bd1
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6488 sha256=efc0f5057e4182f51c9c8302e21b289ff324784e971d01165784fb54b267d427
  Stored in directory: /home/ec2-user/.cache/pip/wheels/44/3a/ab/102386d84fe551b6cedb628ed1e74c5f5be76af8b909aeda09
Successfully built promise subprocess32


In [9]:
!python ../train_dalle.py --image_text_folder ../images --taming

using pretrained VAE for encoding images to tokens
  2%|▌                        | 20725760/957954257 [04:41<3:32:07, 73639.85it/s]
Traceback (most recent call last):
  File "../train_dalle.py", line 207, in <module>
    vae = vae_klass()
  File "/home/ec2-user/SageMaker/DALLE-pytorch/dalle_pytorch/vae.py", line 140, in __init__
    download(VQGAN_VAE_PATH, model_filename)
  File "/home/ec2-user/SageMaker/DALLE-pytorch/dalle_pytorch/vae.py", line 81, in download
    buffer = source.read(8192)
  File "/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p36/lib/python3.6/http/client.py", line 463, in read
    n = self.readinto(b)
  File "/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p36/lib/python3.6/http/client.py", line 507, in readinto
    n = self.fp.readinto(b)
  File "/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p36/lib/python3.6/socket.py", line 586, in readinto
    return self._sock.recv_into(b)
  File "/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p36/